In [1]:
from functions import Model, get99, reg_train_test, splitData, dropMissingValues, z_score, outliers_knn, outliers_dbscan, getNoise, getRelFeatures, drop_features, getCombinations, get_unique_list
import pandas as pd
from sklearn.linear_model import LinearRegression
import warnings
warnings.filterwarnings('ignore')

In [2]:
SPLIT_SIZE = 0.25

In [3]:
#Get raw Data
df = pd.read_csv("data/csv/house_data_training.csv", sep=';') 
# remove unnamed column
df = df.iloc[:, 1:]
#Transform string to datetime
df["date"] = pd.to_datetime(df["date"])

maeList = []#for model selection

In [4]:
df = dropMissingValues(df) 

In [5]:
# Outlier Detection
outlier_list_99 = get99(df)
outlier_list_z_score = z_score(df, 4)
outlier_list_knn = outliers_knn(df, 3, 200) #181
outlier_list_dbscan = outliers_dbscan(df)
outlier_list_noise = getNoise(df)

#TODO: Create list_all from ultiple parameters

outlier_lists_all = [outlier_list_99, outlier_list_z_score, outlier_list_knn, outlier_list_dbscan, outlier_list_noise]

In [6]:
combination_list = []
for i, combo in enumerate(getCombinations(range(len(outlier_lists_all))), 1):
    if not i == 1:
        combination_list.append(combo)
        print('combo #{}: {}'.format(i, combo))


combo #2: (0,)
combo #3: (1,)
combo #4: (2,)
combo #5: (3,)
combo #6: (4,)
combo #7: (0, 1)
combo #8: (0, 2)
combo #9: (0, 3)
combo #10: (0, 4)
combo #11: (1, 2)
combo #12: (1, 3)
combo #13: (1, 4)
combo #14: (2, 3)
combo #15: (2, 4)
combo #16: (3, 4)
combo #17: (0, 1, 2)
combo #18: (0, 1, 3)
combo #19: (0, 1, 4)
combo #20: (0, 2, 3)
combo #21: (0, 2, 4)
combo #22: (0, 3, 4)
combo #23: (1, 2, 3)
combo #24: (1, 2, 4)
combo #25: (1, 3, 4)
combo #26: (2, 3, 4)
combo #27: (0, 1, 2, 3)
combo #28: (0, 1, 2, 4)
combo #29: (0, 1, 3, 4)
combo #30: (0, 2, 3, 4)
combo #31: (1, 2, 3, 4)
combo #32: (0, 1, 2, 3, 4)


In [7]:
t = [[1,2,3], ["a", "b", "c"], [10, 100, 1000, 1]]
clist = []
for i, combo in enumerate(getCombinations(range(len(t))), 1):
    if not i == 1:
        clist.append(combo)
        print('combo #{}: {}'.format(i, combo))


for i, c in enumerate(clist):
    u = get_unique_list(t, c)
    print(u)

# uniques = list(set().union(t[0], t[2]))
# uniques = list(set().union(uniques, t[1]))
# uniques

combo #2: (0,)
combo #3: (1,)
combo #4: (2,)
combo #5: (0, 1)
combo #6: (0, 2)
combo #7: (1, 2)
combo #8: (0, 1, 2)
(0,)
[1, 2, 3]
(1,)
['a', 'b', 'c']
(2,)
[10, 100, 1000, 1]
(0, 1)
['c', 1, 2, 3, 'a', 'b']
(0, 2)
[1, 2, 3, 100, 1000, 10]
(1, 2)
['c', 1, 100, 1000, 10, 'a', 'b']
(0, 1, 2)
['c', 1, 2, 3, 100, 1000, 10, 'a', 'b']


In [8]:
model_obj_list = []
for i, c in enumerate(combination_list):
    outlier_list = get_unique_list(outlier_lists_all, c)
    print(len(outlier_list))
    try: 
        X_train, X_test, y_train, y_test = splitData(df, SPLIT_SIZE, outlier_list)
        #model = reg_train_test(X_train, X_test, y_train, y_test)
        model = LinearRegression()
        obj = Model(model, (X_train, X_test, y_train, y_test), c)
        model_obj_list.append(obj)
    except:
        print("Error in Combination: ", c)
        pass

(0,)
432
dropped 316 rows
(1,)
1013
dropped 755 rows
(2,)
200
Error in Combination:  (2,)
(3,)
2762
dropped 2582 rows
(4,)
427
dropped 313 rows
(0, 1)
763
dropped 763 rows
(0, 2)
513
dropped 499 rows
(0, 3)
2832
dropped 2832 rows
(0, 4)
321
dropped 321 rows
(1, 2)
948
dropped 934 rows
(1, 3)
3175
dropped 3175 rows
(1, 4)
760
dropped 760 rows
(2, 3)
2596
dropped 2582 rows
(2, 4)
510
dropped 496 rows
(3, 4)
2831
dropped 2831 rows
(0, 1, 2)
956
dropped 942 rows
(0, 1, 3)
3181
dropped 3181 rows
(0, 1, 4)
768
dropped 768 rows
(0, 2, 3)
2846
dropped 2832 rows
(0, 2, 4)
518
dropped 504 rows
(0, 3, 4)
2837
dropped 2837 rows
(1, 2, 3)
3189
dropped 3175 rows
(1, 2, 4)
953
dropped 939 rows
(1, 3, 4)
3180
dropped 3180 rows
(2, 3, 4)
2845
dropped 2831 rows
(0, 1, 2, 3)
3195
dropped 3181 rows
(0, 1, 2, 4)
961
dropped 947 rows
(0, 1, 3, 4)
3186
dropped 3186 rows
(0, 2, 3, 4)
2851
dropped 2837 rows
(1, 2, 3, 4)
3194
dropped 3180 rows
(0, 1, 2, 3, 4)
3200
dropped 3186 rows


In [9]:
#Feature Selection
list_relevant_features = getRelFeatures(df)
#df_dropped_features = drop_features(df, list_relevant_features)

In [18]:
#Summary
df_summary = pd.DataFrame(columns=['combo', 'mae', 'score'])
for o in model_obj_list:
    #print(f'{o.get_type()}, {o.get_comb()}')
    o.train()
    mae, score = o.summary()
    comb = o.get_comb()
    row = {'combo': comb, 'mae': mae, 'score': score}
    df_summary = df_summary.append(row, ignore_index=True)
#row
df_summary.sort_values("mae")
#df_summary["mae"].min()



,combo,mae,score
0,"(0,)",6283726.02,0.0495
3,"(4,)",6284195.62,0.0495
7,"(0, 4)",6286589.37,0.0495
5,"(0, 2)",6304853.08,0.0494
12,"(2, 4)",6305358.26,0.0494
18,"(0, 2, 4)",6307791.71,0.0494
1,"(1,)",6351758.08,0.0494
4,"(0, 1)",6354252.36,0.0494
10,"(1, 4)",6354786.13,0.0494
16,"(0, 1, 4)",6357286.16,0.0494
